In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import random
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D

2024-09-01 22:59:32.379461: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-01 22:59:32.421950: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-01 22:59:32.422931: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-01 22:59:33.092482: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
train_dir='dataset/train'
test_dir='dataset/test'

In [3]:
def load_dataset(directory):
  image_paths=[]
  labels=[]

  for label in os.listdir(directory):
    for filename in os.listdir(directory+"/"+label):
      image_path=os.path.join(directory,label,filename)
      image_paths.append(image_path)
      labels.append(label)
    print(label,"compelted")

  return image_paths, labels

In [4]:
train=pd.DataFrame()
train['image'], train['label']=load_dataset(train_dir)

train=train.sample(frac=1).reset_index(drop=True)
train.head()

fear compelted
disgust compelted
neutral compelted
surprise compelted
sad compelted
happy compelted
angry compelted


,image,label
0,dataset/train/surprise/Training_75439903.jpg,surprise
1,dataset/train/sad/Training_2155073.jpg,sad
2,dataset/train/surprise/Training_646132.jpg,surprise
3,dataset/train/happy/Training_86017612.jpg,happy
4,dataset/train/happy/Training_94248483.jpg,happy


In [5]:
test=pd.DataFrame()
test['image'], test['label']=load_dataset(test_dir)
test.head()

fear compelted
disgust compelted
neutral compelted
surprise compelted
sad compelted
happy compelted
angry compelted


,image,label
0,dataset/test/fear/PublicTest_80861502.jpg,fear
1,dataset/test/fear/PrivateTest_7183558.jpg,fear
2,dataset/test/fear/PublicTest_44296746.jpg,fear
3,dataset/test/fear/PublicTest_65218414.jpg,fear
4,dataset/test/fear/PublicTest_19459861.jpg,fear


In [6]:
def extract_features(images):
  features=[]
  for image in tqdm(images):
    img=load_img(image,grayscale=True)
    img=np.array(img)
    features.append(img)
  features=np.array(features)
  features=features.reshape(len(features),48,48,1)
  return features


In [7]:
train_features=extract_features(train['image'])

  0%|          | 0/28709 [00:00<?, ?it/s]

In [8]:
test_features=extract_features(test['image'])

  0%|          | 0/7178 [00:00<?, ?it/s]

In [9]:
x_train=train_features/255.0
x_test=test_features/255.0

In [10]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
le.fit(train['label'])
y_train=le.transform(train['label'])
y_test=le.transform(test['label'])

In [11]:
y_train=to_categorical(y_train,num_classes=7)
y_test=to_categorical(y_test, num_classes=7)

In [12]:
y_train[1]

array([0., 0., 0., 0., 0., 1., 0.], dtype=float32)

In [13]:
input_shape=(48,48,1)
output_class=7


In [14]:
from keras.models import load_model
model1=load_model('Model1_2_sigmoid.h5')
model2=load_model('model2.h5')
model3=load_model('model3_sep1.h5')

In [15]:
predictions_model1=model1.predict(x_train)


2024-09-01 22:59:38.643046: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 264582144 exceeds 10% of free system memory.


898/898 [==============================] - 57s 63ms/step


In [16]:
predictions_model2=model2.predict(x_train)

2024-09-01 23:00:36.565717: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 264582144 exceeds 10% of free system memory.


898/898 [==============================] - 91s 101ms/step


In [17]:
predictions_model3=model3.predict(x_train)

  1/898 [..............................] - ETA: 1:45

2024-09-01 23:02:08.229164: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 264582144 exceeds 10% of free system memory.


898/898 [==============================] - 36s 40ms/step


In [18]:
type(predictions_model3)

numpy.ndarray

In [19]:
predictions_model1[0]

array([0.76844704, 0.24443908, 0.91646063, 0.147187  , 0.03637497,
       0.20393467, 0.9355649 ], dtype=float32)

In [20]:
predictions_model2[0]

array([0.90087646, 0.05616914, 0.88640666, 0.09395977, 0.06034727,
       0.02598925, 0.9980709 ], dtype=float32)

In [21]:
predictions_model3[0]

array([5.1594064e-02, 2.0950129e-06, 1.7006189e-01, 1.4523627e-03,
       1.3015651e-04, 4.5334407e-05, 5.3693724e-01], dtype=float32)

In [22]:
predictions=predictions_model1+predictions_model2+predictions_model3

In [23]:
predictions[0]

array([1.7209175 , 0.3006103 , 1.9729292 , 0.24259913, 0.0968524 ,
       0.22996925, 2.470573  ], dtype=float32)

In [24]:
predictions=predictions/3

In [25]:
predictions[0]

array([0.57363915, 0.10020343, 0.6576431 , 0.08086637, 0.03228413,
       0.07665642, 0.8235243 ], dtype=float32)

In [26]:
true_class=[]
for i in y_train:
    j=i.tolist()
    true_class.append(j.index(1))


In [27]:
true_class=np.array(true_class)

In [28]:
y_features1 = model1.predict(x_test)

225/225 [==============================] - 14s 64ms/step


In [29]:
y_features2=model2.predict(x_test)

225/225 [==============================] - 23s 100ms/step


In [30]:
y_features3=model3.predict(x_test)

225/225 [==============================] - 9s 42ms/step


In [31]:
y_features=y_features1+y_features2+y_features3

In [32]:
y_features=y_features/3

In [33]:
y_features[0]

array([0.5271755 , 0.01195352, 0.7871284 , 0.33555272, 0.16181397,
       0.38867188, 0.6290972 ], dtype=float32)

In [34]:
valid_class=[]
for i in y_test:
    j=i.tolist()
    valid_class.append(j.index(1))
test_answers=np.array(valid_class)

In [35]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score,accuracy_score
predicted_classes = np.argmax(y_features1, axis=1)
# Calculate confusion matrix
cm = confusion_matrix(test_answers, predicted_classes)
print("Confusion Matrix:")
print(cm)

# Optional: Calculate accuracy score
accuracy = accuracy_score(test_answers, predicted_classes)
print("Accuracy Score:", accuracy)

f1 = f1_score(test_answers, predicted_classes,average='macro')
print(f1)

Confusion Matrix:
[[ 331    9   66   30  156  336   30]
 [  20   31    7    3    5   42    3]
 [  55    2  271   19  115  426  136]
 [  16    0   57 1205  223  212   61]
 [  17    0   30   32  785  351   18]
 [  36    0   54   29  235  874   19]
 [  14    0   58   13   44   54  648]]
Accuracy Score: 0.5774589022011702
0.5402956205134447


In [36]:
predicted_classes = np.argmax(y_features2, axis=1)
# Calculate confusion matrix
cm = confusion_matrix(test_answers, predicted_classes)
print("Confusion Matrix:")
print(cm)

# Optional: Calculate accuracy score
accuracy = accuracy_score(test_answers, predicted_classes)
print("Accuracy Score:", accuracy)

f1 = f1_score(test_answers, predicted_classes,average='macro')
print(f1)

Confusion Matrix:
[[ 528   22   53   76  140  116   23]
 [  20   70    2    4    3    9    3]
 [ 165   12  287   64  182  191  123]
 [  51    2   21 1483  143   50   24]
 [ 114    5   26  104  825  145   14]
 [ 160    9   66  108  327  557   20]
 [  30    2   34   56   55   21  633]]
Accuracy Score: 0.6106157704095848
0.587395690002925


In [37]:
predicted_classes = np.argmax(y_features3, axis=1)
# Calculate confusion matrix
cm = confusion_matrix(test_answers, predicted_classes)
print("Confusion Matrix:")
print(cm)

# Optional: Calculate accuracy score
accuracy = accuracy_score(test_answers, predicted_classes)
print("Accuracy Score:", accuracy)

f1 = f1_score(test_answers, predicted_classes,average='macro')
print(f1)

Confusion Matrix:
[[ 545   13   67   49  120  147   17]
 [  28   61    5    2    4   11    0]
 [ 121    1  387   38  123  238  116]
 [  37    0   26 1527  106   44   34]
 [  70    0   60   73  809  205   16]
 [ 109    3  130   67  234  687   17]
 [  17    0   71   45   18    9  671]]
Accuracy Score: 0.6529674003900808
0.635137131082849


In [38]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy')
classifier.fit(predictions_model1, true_class)

RandomForestClassifier(criterion='entropy')

In [39]:
y_pred = classifier.predict(y_features1)

In [40]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 537   13   77   66  107  142   16]
 [  31   56    2    4    3   13    2]
 [ 123    7  449   43  110  184  108]
 [  69    0   64 1428  102   73   38]
 [ 123    1   77  101  698  217   16]
 [ 146    1  152   79  220  635   14]
 [  28    3   85   40   33   11  631]]


0.6177208135971023

In [41]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
precision = precision_score(test_answers, y_pred,average='macro')
recall = recall_score(test_answers, y_pred,average='macro')
f1 = f1_score(test_answers, y_pred,average='macro')
print(precision)
print(recall)
print(f1)

0.616726616245035
0.5918759425066332
0.6017500743738051


In [42]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(predictions_model1, true_class)

SVC(kernel='linear', random_state=0)

In [43]:
y_pred = classifier.predict(y_features1)

In [44]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 537   13   77   66  107  142   16]
 [  31   56    2    4    3   13    2]
 [ 123    7  449   43  110  184  108]
 [  69    0   64 1428  102   73   38]
 [ 123    1   77  101  698  217   16]
 [ 146    1  152   79  220  635   14]
 [  28    3   85   40   33   11  631]]


0.5973808860406798

In [45]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
precision = precision_score(test_answers, y_pred,average='macro')
recall = recall_score(test_answers, y_pred,average='macro')
f1 = f1_score(test_answers, y_pred,average='macro')
print(precision)
print(recall)
print(f1)

0.5769671564854869
0.5633313550149797
0.5628591656787819


In [46]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(predictions_model1, true_class)

SVC(random_state=0)

In [47]:
y_pred = classifier.predict(y_features1)

In [48]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 537   13   77   66  107  142   16]
 [  31   56    2    4    3   13    2]
 [ 123    7  449   43  110  184  108]
 [  69    0   64 1428  102   73   38]
 [ 123    1   77  101  698  217   16]
 [ 146    1  152   79  220  635   14]
 [  28    3   85   40   33   11  631]]


0.6032320980774589

In [49]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
precision = precision_score(test_answers, y_pred,average='macro')
recall = recall_score(test_answers, y_pred,average='macro')
f1 = f1_score(test_answers, y_pred,average='macro')
print(precision)
print(recall)
print(f1)

0.586600377107391
0.5664198667481221
0.5716921923192124


In [50]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(predictions_model1, true_class)

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [51]:
y_pred = classifier.predict(y_features1)

In [52]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 537   13   77   66  107  142   16]
 [  31   56    2    4    3   13    2]
 [ 123    7  449   43  110  184  108]
 [  69    0   64 1428  102   73   38]
 [ 123    1   77  101  698  217   16]
 [ 146    1  152   79  220  635   14]
 [  28    3   85   40   33   11  631]]


0.5546113123432711

In [53]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
precision = precision_score(test_answers, y_pred,average='macro')
recall = recall_score(test_answers, y_pred,average='macro')
f1 = f1_score(test_answers, y_pred,average='macro')
print(precision)
print(recall)
print(f1)

0.5356621133711036
0.5378333100451644
0.5365750710326924


In [54]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 20, metric = 'minkowski', p = 2)
classifier.fit(predictions_model1, true_class)

KNeighborsClassifier(n_neighbors=20)

In [55]:
y_pred = classifier.predict(y_features1)

In [56]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 537   13   77   66  107  142   16]
 [  31   56    2    4    3   13    2]
 [ 123    7  449   43  110  184  108]
 [  69    0   64 1428  102   73   38]
 [ 123    1   77  101  698  217   16]
 [ 146    1  152   79  220  635   14]
 [  28    3   85   40   33   11  631]]


0.589997213708554

In [57]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
precision = precision_score(test_answers, y_pred,average='macro')
recall = recall_score(test_answers, y_pred,average='macro')
f1 = f1_score(test_answers, y_pred,average='macro')
print(precision)
print(recall)
print(f1)

0.5744274800144259
0.5560298083196676
0.5624348800872291


In [58]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(predictions_model1, true_class)

LogisticRegression(random_state=0)

In [59]:
y_pred = classifier.predict(y_features1)

In [60]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 537   13   77   66  107  142   16]
 [  31   56    2    4    3   13    2]
 [ 123    7  449   43  110  184  108]
 [  69    0   64 1428  102   73   38]
 [ 123    1   77  101  698  217   16]
 [ 146    1  152   79  220  635   14]
 [  28    3   85   40   33   11  631]]


0.5962663694622458

In [61]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
precision = precision_score(test_answers, y_pred,average='macro')
recall = recall_score(test_answers, y_pred,average='macro')
f1 = f1_score(test_answers, y_pred,average='macro')
print(precision)
print(recall)
print(f1)

0.5807156736494674
0.5588450736197549
0.5647315365786956


In [62]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(predictions_model1, true_class)

GaussianNB()

In [63]:
y_pred = classifier.predict(y_features1)

In [64]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 537   13   77   66  107  142   16]
 [  31   56    2    4    3   13    2]
 [ 123    7  449   43  110  184  108]
 [  69    0   64 1428  102   73   38]
 [ 123    1   77  101  698  217   16]
 [ 146    1  152   79  220  635   14]
 [  28    3   85   40   33   11  631]]


0.548202842017275

In [65]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
precision = precision_score(test_answers, y_pred,average='macro')
recall = recall_score(test_answers, y_pred,average='macro')
f1 = f1_score(test_answers, y_pred,average='macro')
print(precision)
print(recall)
print(f1)

0.5133106995664912
0.5289381000465049
0.5098585168350758


In [66]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy')
classifier.fit(predictions_model2, true_class)

RandomForestClassifier(criterion='entropy')

In [67]:
y_pred = classifier.predict(y_features2)

In [68]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 537   13   77   66  107  142   16]
 [  31   56    2    4    3   13    2]
 [ 123    7  449   43  110  184  108]
 [  69    0   64 1428  102   73   38]
 [ 123    1   77  101  698  217   16]
 [ 146    1  152   79  220  635   14]
 [  28    3   85   40   33   11  631]]


0.620785734187796

In [69]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
precision = precision_score(test_answers, y_pred,average='macro')
recall = recall_score(test_answers, y_pred,average='macro')
f1 = f1_score(test_answers, y_pred,average='macro')
print(precision)
print(recall)
print(f1)

0.628501741490388
0.6009767030578742
0.612397413188052


In [70]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(predictions_model2, true_class)

SVC(kernel='linear', random_state=0)

In [71]:
y_pred = classifier.predict(y_features2)

In [72]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 537   13   77   66  107  142   16]
 [  31   56    2    4    3   13    2]
 [ 123    7  449   43  110  184  108]
 [  69    0   64 1428  102   73   38]
 [ 123    1   77  101  698  217   16]
 [ 146    1  152   79  220  635   14]
 [  28    3   85   40   33   11  631]]


0.6065756478127612

In [73]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
precision = precision_score(test_answers, y_pred,average='macro')
recall = recall_score(test_answers, y_pred,average='macro')
f1 = f1_score(test_answers, y_pred,average='macro')
print(precision)
print(recall)
print(f1)

0.5980354254891804
0.5778943191723128
0.5859390269767614


In [74]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(predictions_model2, true_class)

SVC(random_state=0)

In [75]:
y_pred = classifier.predict(y_features2)

In [76]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 537   13   77   66  107  142   16]
 [  31   56    2    4    3   13    2]
 [ 123    7  449   43  110  184  108]
 [  69    0   64 1428  102   73   38]
 [ 123    1   77  101  698  217   16]
 [ 146    1  152   79  220  635   14]
 [  28    3   85   40   33   11  631]]


0.6078294789634996

In [77]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
precision = precision_score(test_answers, y_pred,average='macro')
recall = recall_score(test_answers, y_pred,average='macro')
f1 = f1_score(test_answers, y_pred,average='macro')
print(precision)
print(recall)
print(f1)

0.6035686783778369
0.5784164363167837
0.5887650190813013


In [78]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(predictions_model2, true_class)

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [79]:
y_pred = classifier.predict(y_features2)

In [80]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 537   13   77   66  107  142   16]
 [  31   56    2    4    3   13    2]
 [ 123    7  449   43  110  184  108]
 [  69    0   64 1428  102   73   38]
 [ 123    1   77  101  698  217   16]
 [ 146    1  152   79  220  635   14]
 [  28    3   85   40   33   11  631]]


0.5952911674561159

In [81]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
precision = precision_score(test_answers, y_pred,average='macro')
recall = recall_score(test_answers, y_pred,average='macro')
f1 = f1_score(test_answers, y_pred,average='macro')
print(precision)
print(recall)
print(f1)

0.6047726707844079
0.5789008943720674
0.5898278486420949


In [82]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 20, metric = 'minkowski', p = 2)
classifier.fit(predictions_model2, true_class)

KNeighborsClassifier(n_neighbors=20)

In [83]:
y_pred = classifier.predict(y_features2)

In [84]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 537   13   77   66  107  142   16]
 [  31   56    2    4    3   13    2]
 [ 123    7  449   43  110  184  108]
 [  69    0   64 1428  102   73   38]
 [ 123    1   77  101  698  217   16]
 [ 146    1  152   79  220  635   14]
 [  28    3   85   40   33   11  631]]


0.5862357202563389

In [85]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
precision = precision_score(test_answers, y_pred,average='macro')
recall = recall_score(test_answers, y_pred,average='macro')
f1 = f1_score(test_answers, y_pred,average='macro')
print(precision)
print(recall)
print(f1)

0.57421137459067
0.5571051719884927
0.5638400717197615


In [86]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(predictions_model2, true_class)

LogisticRegression(random_state=0)

In [87]:
y_pred = classifier.predict(y_features2)

In [88]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 537   13   77   66  107  142   16]
 [  31   56    2    4    3   13    2]
 [ 123    7  449   43  110  184  108]
 [  69    0   64 1428  102   73   38]
 [ 123    1   77  101  698  217   16]
 [ 146    1  152   79  220  635   14]
 [  28    3   85   40   33   11  631]]


0.6067149623850655

In [89]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
precision = precision_score(test_answers, y_pred,average='macro')
recall = recall_score(test_answers, y_pred,average='macro')
f1 = f1_score(test_answers, y_pred,average='macro')
print(precision)
print(recall)
print(f1)

0.6076639936358851
0.5793110936772472
0.5905133590865572


In [90]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(predictions_model2, true_class)

GaussianNB()

In [91]:
y_pred = classifier.predict(y_features2)

In [92]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 537   13   77   66  107  142   16]
 [  31   56    2    4    3   13    2]
 [ 123    7  449   43  110  184  108]
 [  69    0   64 1428  102   73   38]
 [ 123    1   77  101  698  217   16]
 [ 146    1  152   79  220  635   14]
 [  28    3   85   40   33   11  631]]


0.5756478127612148

In [93]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
precision = precision_score(test_answers, y_pred,average='macro')
recall = recall_score(test_answers, y_pred,average='macro')
f1 = f1_score(test_answers, y_pred,average='macro')
print(precision)
print(recall)
print(f1)

0.54163126758143
0.5663917161602805
0.5469158281035382


In [94]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy')
classifier.fit(predictions_model3, true_class)

RandomForestClassifier(criterion='entropy')

In [95]:
y_pred = classifier.predict(y_features3)

In [96]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 537   13   77   66  107  142   16]
 [  31   56    2    4    3   13    2]
 [ 123    7  449   43  110  184  108]
 [  69    0   64 1428  102   73   38]
 [ 123    1   77  101  698  217   16]
 [ 146    1  152   79  220  635   14]
 [  28    3   85   40   33   11  631]]


0.6489272777932572

In [97]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
precision = precision_score(test_answers, y_pred,average='macro')
recall = recall_score(test_answers, y_pred,average='macro')
f1 = f1_score(test_answers, y_pred,average='macro')
print(precision)
print(recall)
print(f1)

0.6513535999859033
0.6287276304571411
0.6385618468817


In [98]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(predictions_model3, true_class)

SVC(kernel='linear', random_state=0)

In [99]:
y_pred = classifier.predict(y_features3)

In [100]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 537   13   77   66  107  142   16]
 [  31   56    2    4    3   13    2]
 [ 123    7  449   43  110  184  108]
 [  69    0   64 1428  102   73   38]
 [ 123    1   77  101  698  217   16]
 [ 146    1  152   79  220  635   14]
 [  28    3   85   40   33   11  631]]


0.6179994427417108

In [101]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
precision = precision_score(test_answers, y_pred,average='macro')
recall = recall_score(test_answers, y_pred,average='macro')
f1 = f1_score(test_answers, y_pred,average='macro')
print(precision)
print(recall)
print(f1)

0.6129120871932839
0.5903016328181762
0.5959809201465285


In [102]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(predictions_model3, true_class)

SVC(random_state=0)

In [103]:
y_pred = classifier.predict(y_features3)

In [104]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 537   13   77   66  107  142   16]
 [  31   56    2    4    3   13    2]
 [ 123    7  449   43  110  184  108]
 [  69    0   64 1428  102   73   38]
 [ 123    1   77  101  698  217   16]
 [ 146    1  152   79  220  635   14]
 [  28    3   85   40   33   11  631]]


0.6205071050431875

In [105]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
precision = precision_score(test_answers, y_pred,average='macro')
recall = recall_score(test_answers, y_pred,average='macro')
f1 = f1_score(test_answers, y_pred,average='macro')
print(precision)
print(recall)
print(f1)

0.6198543296099646
0.591004188628152
0.6016285923492525


In [106]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(predictions_model3, true_class)

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [107]:
y_pred = classifier.predict(y_features3)

In [108]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 537   13   77   66  107  142   16]
 [  31   56    2    4    3   13    2]
 [ 123    7  449   43  110  184  108]
 [  69    0   64 1428  102   73   38]
 [ 123    1   77  101  698  217   16]
 [ 146    1  152   79  220  635   14]
 [  28    3   85   40   33   11  631]]


0.6200891613262747

In [109]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
precision = precision_score(test_answers, y_pred,average='macro')
recall = recall_score(test_answers, y_pred,average='macro')
f1 = f1_score(test_answers, y_pred,average='macro')
print(precision)
print(recall)
print(f1)

0.6252077555695826
0.5982022246419822
0.6095293076195569


In [110]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 20, metric = 'minkowski', p = 2)
classifier.fit(predictions_model3, true_class)

KNeighborsClassifier(n_neighbors=20)

In [111]:
y_pred = classifier.predict(y_features3)

In [112]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 537   13   77   66  107  142   16]
 [  31   56    2    4    3   13    2]
 [ 123    7  449   43  110  184  108]
 [  69    0   64 1428  102   73   38]
 [ 123    1   77  101  698  217   16]
 [ 146    1  152   79  220  635   14]
 [  28    3   85   40   33   11  631]]


0.6134020618556701

In [113]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
precision = precision_score(test_answers, y_pred,average='macro')
recall = recall_score(test_answers, y_pred,average='macro')
f1 = f1_score(test_answers, y_pred,average='macro')
print(precision)
print(recall)
print(f1)

0.6159517465944947
0.5848800777648647
0.5964634565050649


In [114]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(predictions_model3, true_class)

LogisticRegression(random_state=0)

In [115]:
y_pred = classifier.predict(y_features3)

In [116]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 537   13   77   66  107  142   16]
 [  31   56    2    4    3   13    2]
 [ 123    7  449   43  110  184  108]
 [  69    0   64 1428  102   73   38]
 [ 123    1   77  101  698  217   16]
 [ 146    1  152   79  220  635   14]
 [  28    3   85   40   33   11  631]]


0.6178601281694065

In [117]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
precision = precision_score(test_answers, y_pred,average='macro')
recall = recall_score(test_answers, y_pred,average='macro')
f1 = f1_score(test_answers, y_pred,average='macro')
print(precision)
print(recall)
print(f1)

0.6188181112720275
0.5839857456916155
0.5952839857665164


In [118]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(predictions_model3, true_class)

GaussianNB()

In [119]:
y_pred = classifier.predict(y_features3)

In [120]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 537   13   77   66  107  142   16]
 [  31   56    2    4    3   13    2]
 [ 123    7  449   43  110  184  108]
 [  69    0   64 1428  102   73   38]
 [ 123    1   77  101  698  217   16]
 [ 146    1  152   79  220  635   14]
 [  28    3   85   40   33   11  631]]


0.5341320702145445

In [121]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
precision = precision_score(test_answers, y_pred,average='macro')
recall = recall_score(test_answers, y_pred,average='macro')
f1 = f1_score(test_answers, y_pred,average='macro')
print(precision)
print(recall)
print(f1)

0.5343765920953435
0.5331515069362283
0.520528782714964


In [122]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy')
classifier.fit(predictions, true_class)

RandomForestClassifier(criterion='entropy')

In [123]:
y_pred = classifier.predict(y_features)

In [124]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 537   13   77   66  107  142   16]
 [  31   56    2    4    3   13    2]
 [ 123    7  449   43  110  184  108]
 [  69    0   64 1428  102   73   38]
 [ 123    1   77  101  698  217   16]
 [ 146    1  152   79  220  635   14]
 [  28    3   85   40   33   11  631]]


0.6505990526609083

In [125]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
precision = precision_score(test_answers, y_pred,average='macro')
recall = recall_score(test_answers, y_pred,average='macro')
f1 = f1_score(test_answers, y_pred,average='macro')
print(precision)
print(recall)
print(f1)

0.6521635417408697
0.6290034447268747
0.6386961689615952


In [126]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(predictions, true_class)

SVC(kernel='linear', random_state=0)

In [127]:
y_pred = classifier.predict(y_features)

In [128]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 537   13   77   66  107  142   16]
 [  31   56    2    4    3   13    2]
 [ 123    7  449   43  110  184  108]
 [  69    0   64 1428  102   73   38]
 [ 123    1   77  101  698  217   16]
 [ 146    1  152   79  220  635   14]
 [  28    3   85   40   33   11  631]]


0.6451657843410421

In [129]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
precision = precision_score(test_answers, y_pred,average='macro')
recall = recall_score(test_answers, y_pred,average='macro')
f1 = f1_score(test_answers, y_pred,average='macro')
print(precision)
print(recall)
print(f1)

0.6499352137025836
0.6206707526212185
0.6320016796302722


In [130]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(predictions, true_class)

SVC(random_state=0)

In [131]:
y_pred = classifier.predict(y_features)

In [132]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 537   13   77   66  107  142   16]
 [  31   56    2    4    3   13    2]
 [ 123    7  449   43  110  184  108]
 [  69    0   64 1428  102   73   38]
 [ 123    1   77  101  698  217   16]
 [ 146    1  152   79  220  635   14]
 [  28    3   85   40   33   11  631]]


0.6493452215101699

In [133]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
precision = precision_score(test_answers, y_pred,average='macro')
recall = recall_score(test_answers, y_pred,average='macro')
f1 = f1_score(test_answers, y_pred,average='macro')
print(precision)
print(recall)
print(f1)

0.6566597433548466
0.6264827495889095
0.6384134253754344


In [134]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(predictions, true_class)

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [135]:
y_pred = classifier.predict(y_features)

In [136]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 537   13   77   66  107  142   16]
 [  31   56    2    4    3   13    2]
 [ 123    7  449   43  110  184  108]
 [  69    0   64 1428  102   73   38]
 [ 123    1   77  101  698  217   16]
 [ 146    1  152   79  220  635   14]
 [  28    3   85   40   33   11  631]]


0.6136806910002787

In [137]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
precision = precision_score(test_answers, y_pred,average='macro')
recall = recall_score(test_answers, y_pred,average='macro')
f1 = f1_score(test_answers, y_pred,average='macro')
print(precision)
print(recall)
print(f1)

0.6192117662411386
0.5972452636917345
0.6066631756572847


In [138]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 20, metric = 'minkowski', p = 2)
classifier.fit(predictions, true_class)

KNeighborsClassifier(n_neighbors=20)

In [139]:
y_pred = classifier.predict(y_features)

In [140]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 537   13   77   66  107  142   16]
 [  31   56    2    4    3   13    2]
 [ 123    7  449   43  110  184  108]
 [  69    0   64 1428  102   73   38]
 [ 123    1   77  101  698  217   16]
 [ 146    1  152   79  220  635   14]
 [  28    3   85   40   33   11  631]]


0.6401504597380886

In [141]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
precision = precision_score(test_answers, y_pred,average='macro')
recall = recall_score(test_answers, y_pred,average='macro')
f1 = f1_score(test_answers, y_pred,average='macro')
print(precision)
print(recall)
print(f1)

0.6488978979562867
0.6135455601918552
0.6265052860027315


In [142]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(predictions, true_class)

LogisticRegression(random_state=0)

In [143]:
y_pred = classifier.predict(y_features)

In [144]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 537   13   77   66  107  142   16]
 [  31   56    2    4    3   13    2]
 [ 123    7  449   43  110  184  108]
 [  69    0   64 1428  102   73   38]
 [ 123    1   77  101  698  217   16]
 [ 146    1  152   79  220  635   14]
 [  28    3   85   40   33   11  631]]


0.6460016717748677

In [145]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
precision = precision_score(test_answers, y_pred,average='macro')
recall = recall_score(test_answers, y_pred,average='macro')
f1 = f1_score(test_answers, y_pred,average='macro')
print(precision)
print(recall)
print(f1)

0.6524885937317162
0.6203395554798095
0.6326211499470219


In [146]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(predictions, true_class)

GaussianNB()

In [147]:
y_pred = classifier.predict(y_features)

In [148]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(test_answers, y_pred)
print(cm)
accuracy_score(test_answers, y_pred)

[[ 537   13   77   66  107  142   16]
 [  31   56    2    4    3   13    2]
 [ 123    7  449   43  110  184  108]
 [  69    0   64 1428  102   73   38]
 [ 123    1   77  101  698  217   16]
 [ 146    1  152   79  220  635   14]
 [  28    3   85   40   33   11  631]]


0.6019782669267205

In [149]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
precision = precision_score(test_answers, y_pred,average='macro')
recall = recall_score(test_answers, y_pred,average='macro')
f1 = f1_score(test_answers, y_pred,average='macro')
print(precision)
print(recall)
print(f1)

0.588033624034286
0.5937160067657151
0.5847823696791016


In [150]:
predicted_classes = np.argmax(y_features, axis=1)
# Calculate confusion matrix
cm = confusion_matrix(test_answers, predicted_classes)
print("Confusion Matrix:")
print(cm)

# Optional: Calculate accuracy score
accuracy = accuracy_score(test_answers, predicted_classes)
print("Accuracy Score:", accuracy)

f1 = f1_score(test_answers, predicted_classes,average='macro')
print(f1)

Confusion Matrix:
[[ 469    5   65   44  164  196   15]
 [  32   48    5    2    4   19    1]
 [  95    1  382   21  142  272  111]
 [  26    0   44 1382  193   85   44]
 [  38    0   39   40  884  218   14]
 [  64    0   90   35  296  749   13]
 [  12    0   48   35   29   22  685]]
Accuracy Score: 0.6407077180273056
0.6211630842937181
